In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE236951"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE236951"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE236951.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE236951.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE236951.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene based immune profiling of vermiform appendix GALT of adult patients with pancreatic cancer or colon diseases."
!Series_summary	"Differential immune gene expression analysis in appendix GALT between patients with pancreatic cancer versus benign or malignant colon disease using the nanostring ncounter platform."
!Series_overall_design	"In this study the GALT of appendix was analyzed by RNA expression analysis of ~700 immune related genes using the Pan Cancer Immune Panel from nanostring. Non inflamed normal looking FFPE appendix tissue from sixteen pancreatic cancer patients, nine colon cancer patients, and nine benign colon disease patients was used to compare GALT immune profiles among the three groups."
Sample Characteristics Dictionary:
{0: ['disease: Pancreatic ductal adenocarcinoma', 'disease: Colon adenocarcinoma', 'disease: Benign colon disease'], 1: ['neoadjuvant therapy: None', 'neoadjuvant therapy: n.a.'], 2: ['Sex: Male', 'Sex: Fema

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at Series_title and Series_summary, this is clearly gene expression data for immune genes
# The nanostring ncounter platform is for gene expression profiling
is_gene_available = True

# 2.1 Data Availability

# For trait (Pancreatic Cancer):
# Looking at row 0 which contains disease information, we can identify patients with pancreatic cancer vs others
trait_row = 0

# For age:
# Row 3 contains age data
age_row = 3

# For gender:
# Row 2 contains gender (Sex) data
gender_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert disease information to binary trait value (1 for Pancreatic Cancer, 0 for other).
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    disease = parts[1].strip().lower()
    # 1 for Pancreatic Cancer, 0 for others (colon cancer or benign)
    if "pancreatic" in disease:
        return 1
    elif "colon" in disease or "benign" in disease:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age string to numerical value.
    """
    if not isinstance(value, str):
        return None
    
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    age_str = parts[1].strip()
    # Extract numerical value from strings like "83 years"
    try:
        age = int(age_str.split()[0])
        return age
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """
    Convert gender information to binary (0 for female, 1 for male).
    """
    if not isinstance(value, str):
        return None
    
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    gender = parts[1].strip().lower()
    if "female" in gender:
        return 0
    elif "male" in gender:
        return 1
    else:
        return None

# 3. Save Metadata
# Perform initial filtering - we know trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We're going to extract clinical features using the function from the library
    # We'll include all three features: trait, age, and gender
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    print("Preview of processed clinical data:")
    print(preview_df(clinical_features_df))
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of processed clinical data:
{'GSM7587683': [1.0, 83.0, 1.0], 'GSM7587684': [1.0, 64.0, 1.0], 'GSM7587685': [1.0, 59.0, 1.0], 'GSM7587686': [1.0, 64.0, 1.0], 'GSM7587687': [1.0, 72.0, 0.0], 'GSM7587688': [1.0, 72.0, 1.0], 'GSM7587689': [1.0, 89.0, 1.0], 'GSM7587690': [1.0, 59.0, 0.0], 'GSM7587691': [1.0, 64.0, 0.0], 'GSM7587692': [1.0, 82.0, 0.0], 'GSM7587693': [1.0, 75.0, 0.0], 'GSM7587694': [1.0, 61.0, 1.0], 'GSM7587695': [1.0, 59.0, 1.0], 'GSM7587696': [1.0, 68.0, 1.0], 'GSM7587697': [1.0, 49.0, 0.0], 'GSM7587698': [1.0, 71.0, 1.0], 'GSM7587699': [0.0, 68.0, 0.0], 'GSM7587700': [0.0, 58.0, 0.0], 'GSM7587701': [0.0, 76.0, 1.0], 'GSM7587702': [0.0, 67.0, 1.0], 'GSM7587703': [0.0, 52.0, 0.0], 'GSM7587704': [0.0, 57.0, 1.0], 'GSM7587705': [0.0, 72.0, 0.0], 'GSM7587706': [0.0, 59.0, 0.0], 'GSM7587707': [0.0, 53.0, 0.0], 'GSM7587708': [0.0, 95.0, 0.0], 'GSM7587709': [0.0, 53.0, 1.0], 'GSM7587710': [0.0, 55.0, 1.0], 'GSM7587711': [0.0, 43.0, 0.0], 'GSM7587712': [0.0, 71.0, 0.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A2M', 'ABCB1', 'ABL1', 'ADA', 'ADORA2A', 'AICDA', 'AIRE', 'AKT3',
       'ALCAM', 'AMBP', 'AMICA1', 'ANP32B', 'ANXA1', 'APOE', 'APP', 'ARG1',
       'ARG2', 'ATF1', 'ATF2', 'ATG10'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers to determine if they need mapping
# The indices shown are valid human gene symbols (e.g., A2M, ABCB1, ABL1, etc.) 
# These are standard HGNC gene symbols that don't require mapping to another format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data and ensure it's in the right format
clinical_features = pd.read_csv(out_clinical_data_file)
print("Clinical features shape:", clinical_features.shape)

# Reprocess the clinical data from the original source to ensure correct format
def convert_trait(value):
    """Convert disease information to binary trait value (1 for Pancreatic Cancer, 0 for other)."""
    if not isinstance(value, str):
        return None
    
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    disease = parts[1].strip().lower()
    if "pancreatic" in disease:
        return 1
    elif "colon" in disease or "benign" in disease:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age string to numerical value."""
    if not isinstance(value, str):
        return None
    
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    age_str = parts[1].strip()
    try:
        age = int(age_str.split()[0])
        return age
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """Convert gender information to binary (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    gender = parts[1].strip().lower()
    if "female" in gender:
        return 0
    elif "male" in gender:
        return 1
    else:
        return None

# Re-extract clinical features
clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,
    convert_trait=convert_trait,
    age_row=3,
    convert_age=convert_age,
    gender_row=2,
    convert_gender=convert_gender
)

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)
print("Linked data columns:", linked_data.columns[:5].tolist())  # Print first 5 columns

# Find the correct trait column in linked_data
# Typically the first non-demographic column after transposition
trait_column = linked_data.columns[0]  # Assuming it's the first column
print(f"Using '{trait_column}' as the trait column for handling missing values")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_column)

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait_column)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Cohort contains pancreatic cancer patients compared with colon diseases (benign and malignant)."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE236951.csv
Clinical features shape: (3, 34)
Linked data shape: (34, 729)
Linked data columns: ['Pancreatic_Cancer', 'Age', 'Gender', 'A2M', 'ABCB1']
Using 'Pancreatic_Cancer' as the trait column for handling missing values


For the feature 'Pancreatic_Cancer', the least common label is '1.0' with 16 occurrences. This represents 47.06% of the dataset.
The distribution of the feature 'Pancreatic_Cancer' in this dataset is fine.



Quartiles for 'Age':
  25%: 55.5
  50% (Median): 63.5
  75%: 71.75
Min: 43.0
Max: 95.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 17 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Linked data saved to ../../output/preprocess/Pancreatic_Cancer/GSE236951.csv
